# Regularization Techniques: Lasso and Ridge Regression

# Introduction and Dataset

## Background

This tutorial will explore Lasso and Ridge regression methods to model different response variables that are commonly modeled in forestry. These include quadratic mean diameter (QMD), aboveground biomass (AGB), and basal area (BA). The tutorial will employ a suite of input features (i.e., predictor variables) used to estimate the response variables. 

## Tutorial goals

**Goal 1: Develope ridge and lasso regression models for QMD, AGB, and BA using LiDAR and multispectral predictor variables**

**Goal 2: Compare ridge and lasso models for each response variable and choose the best model for each**

**Goal 3: Apply the best performing model for each response variable across the entire PRF**

-----

## Data

Please refer to the README on the main GitHub page for a detailed description of each file.


## Packages

- GeoPandas
- rioxarray
- spyndex

# Install and load packages

**Uncomment the cell below to install required packages**

In [ ]:
# !pip install pandas==2.2.2
# !pip install geopandas==1.0.1
# !pip install matplotlib==3.10.1
# !pip install rioxarray==0.19.0
# !pip install spyndex==0.5.0

In [ ]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import rioxarray as rio
import spyndex
from spyndex import indices
from math import sqrt, pi
import matplotlib.pyplot as plt
from rasterio.plot import show
from matplotlib import pyplot as plt

# Download data

In [ ]:
# Download the data if it does not yet exist
if not os.path.exists("data"):
  !gdown 1UDKAdXW0h6JSf7k31PZ-srrQ3487l9e2
  !unzip prf_data.zip -d data/
  os.remove("prf_data.zip")
else:
  print("Data has already been downloaded.")

os.listdir("data")

# Preprocessing

Before we can begin with ridge and lasso regression, we must first preprocess the data so it is analysis ready. The following code blocks will prepare both the response variables (QMD, AGB, BA) in addition to predictor variables (99th height percentile and spectral indices).

In [ ]:

trees_df = gpd.read_file(r'data/trees.csv')

plots_gdf = gpd.read_file(r'data/plots.gpkg').rename(columns={"Plot": "PlotName"})

print(trees_df.head())

In [ ]:
# Ensure multiple columns in trees df are numeric
cols_to_convert = ['biomass', 'height', 'baha', 'DBH']
for col in cols_to_convert:
    trees_df[col] = pd.to_numeric(trees_df[col])

trees_df

In [ ]:
# Check the range of various tree attributes
trees_df.describe()

## Response Variables

### Quadratic Mean Diameter

Quadratic Mean Diameter (QMD) is a common stand level attribute that is modeled in forestry. QMD is often prefered over the arithmetic mean in forestry because it gives greater weight to larger trees. This is relevant for several reasons, primarily though because the wood from larger trees is more valuable.

QMD also is relevant for understanding forest ecology among other applications.

In [ ]:
# Calculate the Quadratic Mean Diameter (QMD)
qmd_df = (
    trees_df
    .groupby('PlotName')
    .agg(
        n_trees=('DBH', 'count'),
        sum_squares=('DBH', lambda x: (x**2).sum())
    )
    .assign(qmd=lambda df: (df['sum_squares'] / df['n_trees']).apply(sqrt))
    .reset_index()[['PlotName', 'qmd']]
)

print(qmd_df.describe())

# Join with plots GeoDataFrame
plots_gdf = plots_gdf.merge(qmd_df, on='PlotName', how='left')

ax = plots_gdf['qmd'].hist(edgecolor='black', color='green')
ax.set_xlabel('QMD (cm)')
ax.set_ylabel('Number of Plots')
ax.set_title('Distribution of Quadratic Mean Diameter (QMD)')
plt.show()

### Aboveground Biomass (AGB)

Forest aboveground biomass (AGB) is another very common stand attribute to model. Biomass is defined as the living organic materials comprising trees including wood, bark, branches, foliage, etc. AGB is modeled for many different reasons. One relevant application of AGB modelling is for forest carbon projects, since forest aboveground carbon is typically estimated to be ~50% of AGB.

We can calculate plot-level AGB by summing the AGB of all trees in a plot, and then dividing that by the plot area. This is performed in the code below.

In [ ]:
# Note that each plot has a radius of 14.1m (625m^2) 
# We need to convert to hectares, since this is the most common areal unit in forestry.
# There are 10000 m^2 in a hectare, so we divide by 10000.

plot_area_m2 = 625

plot_area_ha = plot_area_m2 / 10000

print(f"Area of each plot in hectares: {plot_area_ha} ha")

# Convert tree-level biomass from Kg/ha to Kg, and then to Mg (tonnes).
trees_df['biomass_kg'] = trees_df['biomass'] * plot_area_ha
trees_df['biomass_Mg'] = trees_df['biomass_kg'] / 1000  

biomass_df = (trees_df.groupby('PlotName').
                    agg(biomass_Mg_total=('biomass_Mg', 'sum')).
                    assign(biomass_Mg_ha=lambda x: x['biomass_Mg_total'] / plot_area_ha))

# Summarize biomass
print(biomass_df.describe())

# Join with plots GeoDataFrame
plots_gdf = plots_gdf.merge(biomass_df, on='PlotName', how='left')

ax = biomass_df['biomass_Mg_total'].hist(edgecolor='black', color='green')
ax.set_xlabel('AGB (Mg/ha)')
ax.set_ylabel('Number of Plots')
ax.set_title('Distribution of Aboveground Biomass')

### Basal Area


In [ ]:
def get_ba(dbh):
    return ((dbh / 2) ** 2) * pi

ba_df = (trees_df
            .assign(ba_cm2=lambda x: get_ba(x['DBH']))
            .assign(ba_m2=lambda x: x['ba_cm2'] / 10000)
            .groupby('PlotName')
            .agg(total_ba_m2_ha=('ba_m2', 'sum'))
            .assign(ba_m2_ha=lambda x: x['total_ba_m2_ha'] / plot_area_ha)
            .reset_index())

ba_df.describe()

# Join with plots GeoDataFrame
plots_gdf = plots_gdf.merge(ba_df, on='PlotName', how='left')

ax = plots_gdf['ba_m2_ha'].hist(edgecolor='black', color='green')
ax.set_xlabel('Basal Area (m2/ha)')
ax.set_ylabel('Number of Plots')
ax.set_title('Distribution of Basal Area (BA)')
plt.show()

## Predictor Variables

### LiDAR-Derived 99th Height Percentile

We load the 9th height percentile raster as an xarray dataset. xarray is similar to numpy arrays, but with added attributes and functionality. For example, xarrays can contain spatial coordinate reference systems (CRS).

In [ ]:
# Read the 99th height percentile raster
p99 = rio.open_rasterio(r'data/p99.tif')
p99

In [ ]:
# Ensure that raster and plot coordinates are in the same CRS
assert plots_gdf.crs == p99.rio.crs, "CRS mismatch between plots and raster data."

In [ ]:
# Get list of plot coordinates tuples
plot_coords = [(geom.x, geom.y) for geom in plots_gdf.geometry]

# Extract the 99th percentile height at the plot coordinates
plots_gdf['p99'] = [p99.sel(x=c[0], y=c[1], method="nearest").values[0] for c in plot_coords]

# We are extracting the LiDAR derived 99th percentile height at the plot coordinates.
fig, ax = plt.subplots(figsize=(15, 15))
show(p99.values, ax=ax, transform=p99.rio.transform())
plots_gdf.plot(ax=ax, color='red', markersize=50, label='Plots')

In [ ]:
# View the distribution of the 99th percentile height
ax = plots_gdf['p99'].hist(edgecolor='black', color='green')
ax.set_xlabel('Height (m)')
ax.set_ylabel('Number of Plots')
ax.set_title('Distribution of 99th Percentile Height')
plt.show()

### Sentinel-2 Spectral Indices

While we can write code to calculate spectral indices, this can become time consuming once we start dealing with many different indices. Moreover, we can make mistakes in our code. As a suitable alternative, the `spyndex` Python package offers a standardized, simpler method for calculating many spectral indices at once.

Read the spyndex documentation here: [https://spyndex.readthedocs.io/en/stable/](https://spyndex.readthedocs.io/en/stable/)

In [ ]:
# Load the Sentinel-2 imagery for 2018 (year the plots were sampled)
s2 = rio.open_rasterio(r'data/petawawa_s2_2018.tif')

assert plots_gdf.crs == s2.rio.crs, "CRS mismatch between plots and raster data."

# Consult the documentation for the spectral bands:
# https://developers.google.com/earth-engine/datasets/catalog/COPERNICUS_S2_SR_HARMONIZED
s2


In [ ]:
# Check range of reflectance for each band
print("Min reflectance in S2 data:", np.nanmin(s2.values))
print("Max reflectance in S2 data:", np.nanmax(s2.values))

**Consult this table for more details about band abbreviations used for spectral index calculation:**

[https://github.com/awesome-spectral-indices/awesome-spectral-indices?tab=readme-ov-file#expressions](https://github.com/awesome-spectral-indices/awesome-spectral-indices?tab=readme-ov-file#expressions)

In [ ]:

print("Sentinel-2 band names order:", s2.long_name)

print("Spyndex band abbreviations:", spyndex.bands)


In [ ]:
# Make a list of spectral indices to calculate
spec_index_ls = ["NDVI", "NBR", "SAVI", "MSAVI", "DSI", "NDWI", "GLI",  "ND705", "NDREI", "IRECI", "TGI"]

In [ ]:
# One nice thing about spyndex is that it links each spectral index with a publication describing it.
# List all the publications for each index

for si in spec_index_ls:
    print(f"{si}: {indices[si].reference}")

In [ ]:
# Compute all the spectral indices

spec_indeces = spyndex.computeIndex(
    index = spec_index_ls,
    params = {
        "A": s2[0], 
        "B": s2[1],
        "G": s2[2],
        "R": s2[3],
        "RE1": s2[4],
        "RE2": s2[5],
        "RE3": s2[6],
        "N": s2[7],
        "N2": s2[8],
        "WV": s2[9],
        "S1": s2[10],
        "S2": s2[11],
        "L": 1
    }

)

In [ ]:
# Extract spectral indices at the plot coordinates

for si_name in spec_index_ls:

    print(f"Extracting {si_name} values at plot coordinates...")

    si_raster = spec_indeces[spec_indeces.index == si_name]

    plots_gdf[si_name] = [si_raster.sel(x=c[0], y=c[1], method="nearest").values[0] for c in plot_coords]

plots_gdf.head(5)


In [ ]:
# View one of the spectral indices

view_si_nm = "NDVI"

view_si_raster = spec_indeces[spec_indeces.index == si_name]

show(view_si_raster.values[0], cmap='viridis')

In [ ]:
# Make lists of all predictor and response variables

predictor_vars = spec_index_ls + ["p99"]
print("Predictor variables:", predictor_vars)

response_vars = ["biomass_Mg_ha", "ba_m2_ha", "qmd"]
print("Response variables:", response_vars)

In [ ]:
# Remove any nan values present in the predictor or response variables
plots_gdf = plots_gdf.dropna(subset=response_vars + predictor_vars)
plots_gdf.shape

In [ ]:
# Plot scatterplot matrix of all predictor and response variables
pd.plotting.scatter_matrix(
    plots_gdf[predictor_vars + response_vars],
    figsize=(20, 20),
    diagonal='kde',
    alpha=0.5,
    color='green'
)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Lasso
from sklearn.metrics import mean_squared_error, r2_score

# Set target variable

target_var = "biomass_Mg_ha"

# Divide features and targets into separate DataFrames
X = plots_gdf[predictor_vars]
y = plots_gdf[target_var]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Train a lasso regression model with initial alpha=0.1
lasso = Lasso(alpha=0.1, max_iter=10000)
lasso.fit(X_train, y_train)

In [ ]:
y_test_pred = lasso.predict(X_test)

# Calculate R2 and RMSE
r2 = r2_score(y_test, y_test_pred)
rmse = sqrt(mean_squared_error(y_test, y_test_pred))
print(f"R2: {r2:.3f}, RMSE: {rmse:.3f}")

In [ ]:
# View the parameters of the model
print("Lasso coefficients:")
for feature, coef in zip(X.columns, lasso.coef_):
    print(f"{feature}: {coef:.4f}")